## One dimensional example of GAN

This is still work in progress

In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Activation
from keras import losses

#### Parameters

In [ ]:
problem_params = {
    'mean': 1.0,
    'sigma': 2.0, 
    'noise_bound': 8.0
}

generator_hidden_dims = 4
model_params = {
    # generator hidden layer size
    'gen_hid_size': generator_hidden_dims,
    # discriminator needs to be more powerful
    # than the generator in this case
    'discrim_hid_size': 2 * generator_hidden_dims,
    'kernel_init': 'random_normal',
    'batch_size': 5,
    'optim': 'Adam',
    'loss': 'binary_crossentropy'
            }

#### Define data sampling

define data sampling functions:  
**sample_real**: sampling of real data from a Gaussian distribution with given sigma and mean  
**sample_noise**: sampling of noise as the generator distribution

In [ ]:
def sample_real(n, sigma=1.0, mean=0.0):
    return np.sort(np.random.normal(mean, sigma, n))

In [ ]:
def sample_noise(n, bound=8.0):
    #use stratified sampling
    return np.linspace(-bound, bound, n) + \
            np.random.random(n) * 0.01

#### Define generator

Job of generator is to generate synthetic data to mimic real data.  
In this case we use a neural network to simulate the inverse of  
the cumulative density function of a Gaussian Distribution.

In [ ]:
def generator(inputs, hidden_dim, init): 
    # first linear transform without activation
    x = Dense(hidden_dim, kernel_initializer=init, 
              activation='softplus')(inputs)
    # apply nonlinear transform using softplus
    return Dense(1, kernel_initializer=init)(x)

#### Define discriminator

In [ ]:
def discriminator(inputs, hidden_dim, init):
    activ_func = 'relu'
    x = Dense(hidden_dim, kernel_initializer=init, 
              activation=activ_func)(inputs)
    for i in range(2):
        x = Dense(hidden_dim, kernel_initializer=init, 
                  activation=activ_func)(x)     
    return Dense(1, kernel_initializer=init,
                        activation='sigmoid')(x)    

#### Define the GAN model

The model will use binary cross entropy as loss function. The targets will depend on the training phase. Essentially we want to achieve:
* First, we train the discriminator to distinguish between the real and fake inputs. The generator is kept fixed, and we use both real and fake (from generator) inputs in the loss function. Target of the real and fake inputs will be one and zero respectively.
* Second, we train the generator to mimic the real inputs. The discriminator is kept fixed, and we only consider the fake inputs from the generator in the loss function. Target of the fake inputs will be one.

Apparently we cannot freeze layers **after** compilation in Keras. So we will need to create two models using the same layers, but freezing them differently for both training phases. Otherwise, we would have to recompile for each batch iteration.

In [ ]:
def gan_as_discriminator(generator, discriminator):
    out_fake = Activation('linear')(
                    discriminator(generator(generator.inputs)))
    out_real = Activation('linear')(
                    discriminator(discriminator.inputs))
    for layer in generator.layers:
        layer.trainable=False
    for layer in discriminator.layers:
        layer.trainable=True
    #inputs of models are lists
    return Model(inputs=generator.inputs + discriminator.inputs,
                 outputs=[out_fake, out_real])

def gan_as_generator(generator, discriminator):
    out_fake = Activation('linear')(
                    discriminator(generator(generator.inputs)))
    for layer in generator.layers:
        layer.trainable=True
    for layer in discriminator.layers:
        layer.trainable=False
    return Model(inputs=generator.inputs, outputs=out_fake)

In [ ]:
class GAN(object):
    def __init__(self, params):
        self.batch_size = params['batch_size']
        # initialize generator
        self.gen_input = Input(shape=(1,), dtype='float32', 
                               name='gen_input')
        self.generator = Model(inputs=self.gen_input, 
                               outputs=generator(self.gen_input, 
                                           params['gen_hid_size'],
                                           params['kernel_init']))
        # initialize discriminator
        self.discrim_input = Input(shape=(1,), dtype='float32', 
                                   name='discrim_input')
        self.discriminator = Model(inputs=self.discrim_input, 
                                   outputs=discriminator(
                                               self.discrim_input,
                                               params['discrim_hid_size'],
                                               params['kernel_init']))  
        # initialize gan version for training generator
        self.gan_gen = gan_as_generator(self.generator, 
                                        self.discriminator)
        # initialize gan version for training discriminator
        self.gan_discrim = gan_as_discriminator(self.generator, 
                                                self.discriminator)
        # compile the models
        for model in [self.generator, self.discriminator,
                      self.gan_gen, self.gan_discrim]:
            model.compile(optimizer=params['optim'], 
                               loss=params['loss']) 
        
        
    def train_discriminator(self, params):
        # generate batch of real and fake data
        real_data = sample_real(self.batch_size,
                                params['sigma'], 
                                params['mean'])
        noise_data = sample_noise(self.batch_size, 
                                  params['noise_bound'])
        # define target labels for real and noise data
        real_target = np.ones(real_data.shape)
        fake_target = np.zeros(noise_data.shape)
        self.gan_discrim.fit(x=[real_data, noise_data], 
                             y=[real_target, fake_target],
                             batch_size=self.batch_size)
        
     
    def train_generator(self, params):
        noise_data = sample_noise(self.batch_size, 
                                  params['noise_bound'])
        target = np.ones(noise_data.shape)
        
        print(type(noise_data[0]))
        print(type(target[0]))
        
        self.gan_gen.fit(x=noise_data, y=target, 
                         batch_size=self.batch_size)

#### train the GAN

In [ ]:
gan = GAN(model_params)

for i in range(10):
    gan.train_discriminator(problem_params)
    gan.train_generator(problem_params)